In [1]:
import tensorflow as tf
import numpy as np
import pickle
import matplotlib.pyplot as plt
import timeit

In [2]:
training_data=pickle.load(open('4training_data.pkl','rb'))
validation_data=pickle.load(open('4validation_data.pkl','rb'))
testing_data=pickle.load(open('4testing_data.pkl','rb'))

In [3]:
tf.reset_default_graph()

In [4]:
def instintiate_weights(key, shape):
    return tf.get_variable(key,shape=shape,initializer=tf.contrib.layers.variance_scaling_initializer(factor=2.0,mode='FAN_IN',uniform=False,seed=None,dtype=tf.float32))
def instintiate_bias(key, shape):
    return tf.Variable(tf.constant(0.1, dtype=tf.float32, shape=shape))

In [5]:
batch_size = 32
epochs = 100

In [6]:
def alex_net(images, num_classes):

        # Layer 1
        with tf.name_scope("conv1") as scope:
            kernel = instintiate_weights('W1', [11, 11, 3, 96])
            conv = tf.nn.conv2d(images, kernel, [1, 4, 4, 1], padding='SAME')
            biases = instintiate_bias('b1', [96])
            conv1 = tf.nn.relu(tf.nn.bias_add(conv, biases), name=scope)

        with tf.name_scope('pool1') as scope:
            pool1 = tf.nn.max_pool(conv1, [1, 3, 3, 1], [1, 2, 2, 1], padding='SAME', name=scope)

        # Layer 2
        with tf.name_scope("conv2") as scope:
            kernel = instintiate_weights('W2', [5, 5, 96, 256])
            conv = tf.nn.conv2d(pool1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = instintiate_bias('b2', [256])
            conv2 = tf.nn.relu(tf.nn.bias_add(conv, biases), name=scope)

        with tf.name_scope('pool2') as scope:
            pool2 = tf.nn.max_pool(conv2, [1, 3, 3, 1], [1, 2, 2, 1], padding='SAME', name=scope)

        # Layer 3
        with tf.name_scope("conv3") as scope:
            kernel = instintiate_weights('W3', [3, 3, 256, 384])
            conv = tf.nn.conv2d(pool2, kernel, [1, 1, 1, 1], padding='SAME')
            biases = instintiate_bias('b3', [384])
            conv3 = tf.nn.relu(tf.nn.bias_add(conv, biases), name=scope)
        # Layer 4
        with tf.name_scope("conv4") as scope:
            kernel = instintiate_weights('W4', [3, 3, 384, 384])
            conv = tf.nn.conv2d(conv3, kernel, [1, 1, 1, 1], padding='SAME')
            biases = instintiate_bias('b4', [384])
            conv4 = tf.nn.relu(tf.nn.bias_add(conv, biases), name=scope)

        # Layer 5
        with tf.name_scope("conv5") as scope:
            kernel = instintiate_weights('W5', [3, 3, 384, 256])
            conv = tf.nn.conv2d(conv4, kernel, [1, 1, 1, 1], padding='SAME')
            biases = instintiate_bias('b5', [256])
            conv5 = tf.nn.relu(tf.nn.bias_add(conv, biases), name=scope)

        with tf.name_scope('pool5') as scope:
            pool5 = tf.nn.max_pool(conv5, [1, 3, 3, 1], [1, 2, 2, 1], padding='SAME', name=scope)

        # Fully Connected 6
        with tf.name_scope('FC6') as scope:
            pool5_flat = tf.contrib.layers.flatten(pool5)
            fc6 = tf.layers.dense(pool5_flat, units=4096, activation=tf.nn.relu, name=scope)
            # fc6_dropout = tf.layers.dropout(fc6, 0.5)

        with tf.name_scope('FC7') as scope:
            fc7 = tf.layers.dense(fc6, units=4096, activation=tf.nn.relu, name=scope)
            # fc7_dropout = tf.layers.dropout(fc7, 0.5)

        with tf.name_scope('classes') as scope:
            predictions = tf.layers.dense(fc7, units=num_classes, activation=tf.nn.softmax, name=scope)

        return predictions

In [7]:
train_images = training_data[0]
train_labels = training_data[1]
valid_images = validation_data[0]
valid_labels = validation_data[1]

num_classes = train_labels.shape[1]
images = tf.placeholder(tf.float32, shape=[None, *train_images[0].shape])
labels = tf.placeholder(tf.float32, shape=[None, 2])

predictions = alex_net(images, num_classes)
init = tf.global_variables_initializer()

loss = tf.nn.l2_loss(tf.square(predictions - labels))
learner = tf.train.GradientDescentOptimizer(0.0001)
optimizer = learner.minimize(loss)

correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess= tf.Session()
sess.run(init)
for _ in range(epochs):
    l_total = 0
    acc = 0
    for i in range(0, len(train_images), batch_size):
        l, acc, _ = sess.run(fetches=[loss, accuracy, optimizer],
                             feed_dict={images: train_images[i: batch_size + i],
                                        labels: train_labels[i: batch_size + i]})
        l_total += np.sum(l)

    print(l_total, acc)
    val_acc, _ = sess.run([accuracy, predictions],
                           feed_dict={images: valid_images, labels: valid_labels})
    print("Validation accuracy: ", val_acc)



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.


KeyboardInterrupt: 

In [7]:
result=sess.run(accuracy,feed_dict={images: test_images,labels: test_labels})

array([[0, 1],
       [1, 0],
       [1, 0],
       ...,
       [1, 0],
       [0, 1],
       [1, 0]])